In [1]:
import importlib.util
import logging
from typing import Any, List, Mapping, Optional
import torch
from pydantic import Extra

from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


class HuggingFacePipeline(LLM):
    pipeline: Any  #: :meta private:
    generation_kwargs: dict

    @property
    def _llm_type(self) -> str:
        return "huggingface_pipeline"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response = self.pipeline(prompt, **self.generation_kwargs)
        if self.pipeline.task == "text-generation":
            # Text generation return includes the starter text.
            text = response[0]["generated_text"][len(prompt) :]
        elif self.pipeline.task == "text2text-generation":
            text = response[0]["generated_text"]
        else:
            raise ValueError(
                f"Got invalid task {self.pipeline.task}, "
            )
        if stop is not None:
            # This is a bit hacky, but I can't figure out a better way to enforce
            # stop tokens when making calls to huggingface_hub.
            text = enforce_stop_tokens(text, stop)
        return text
      

    
model_id = "philschmid/instruct-igel-001"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.bfloat16)
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer
)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
  
generate_kwargs = dict(
    top_p=1,
    top_k=0,
    temperature=0.4,
    do_sample=True,
    max_new_tokens=2048,
    early_stopping=True,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)


llm = HuggingFacePipeline(pipeline=pipe,generation_kwargs=generate_kwargs)

In [3]:
from langchain import PromptTemplate


template = "### Anweisung:\n{input}\n\n### Antwort:"

prompt = PromptTemplate(
    input_variables=["input"],
    template=template,
)

def generate(input_variables: Mapping[str, str],prompt=prompt) -> str:
    return llm(prompt.format(**input_variables)).strip()


generate({"input": "Wie ist das Wetter heute?"})

'Heute ist das Wetter in der Stadt sonnig und warm bei etwa 30 °C.'

In [4]:
from langchain.prompts import PromptTemplate

qa_template = """Beantworten Sie die Frage am Ende des Textes anhand der folgenden Informationen. Wenn Sie die Antwort nicht wissen, sagen Sie einfach, dass Sie es nicht wissen, versuchen Sie nicht, eine Antwort zu erfinden.

{context}

Frage: {question}"""
qa_prompt = PromptTemplate(
    template=qa_template, input_variables=["context", "question"]
)


r = generate({"context": "Olaf Scholz ist aktueller Bundeskanzler", "question": "Wer ist aktueller Kanzler?"}, prompt=qa_prompt)
print(r)

Antwort: Olaf Scholz.


In [12]:
import os
from langchain.utilities import GoogleSerperAPIWrapper

os.environ["SERPER_API_KEY"] = "0605930b7bcc70497b8d0a6a37dad17ec000f837"

search = GoogleSerperAPIWrapper(k=3,gl="de",hl="de")

# search.run("Wie ist das Wetter in Berlin?")
# print(search.run("Wo steht der Dax?"))

def google_search(input):
  r = search.run(input)
  # print(qa_prompt.format(context=r, question=input))
  o = generate({"context": r, "question": input}, prompt=qa_prompt)
  if "Antwort:" in o:
    o = o.split("Antwort:")[-1]
  return o

  

In [13]:
print(google_search("Wer ist aktueller Direktor des ITZBund?"))

Antworten:

- Seit Juli 2017 ist Dr. Alfred Kranstedt Direktor des ITZBund.

- Im November 2020 wurde Dr. Kranstedt in den Vorstand von NExT – Netzwerk: Experten für die digitale Transformation der Verwaltung, Direktorium, gewählt.

- Dr. Alfred Kranstedt, Jahrgang 1966; Diplom-Informatiker (Universität Bielefeld) ; Direktor des Kaufmännischen Vizedirektors des ITZBund.

- Seit Juli 2017 Direktor des ITZBund

- Seit November 2020 Vorstandsmitglied bei NExT – Netzwerk: Experten für die digitale Transformation der Verwaltung, Direktorium, gewählt.

- Dr. Alfred Kranstedt, Jahrgang 1966; Diplom-Informatiker (Universität Bielefeld) ; Direktor des Kaufmännischen Vizedirektors des ITZBund.


In [17]:
print(google_search("Wie macht ein Schwein?"))

/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Antworten:

1. Grunzen
2. Krähen
3. Knurren
4. Grunzen
5. Schnauben
6. Grunzen
7. Grunzen
8. Grunzen
9. Grunzen
10. Grunzen
11. Grunzen
12. Grunzen
13. Grunzen
14. Grunzen
15. Grunzen
16. Grunzen
17. Grunzen
18. Grunzen
19. Grunzen
